## 머신러닝의 모델평가
* 일반적으로 머신러닝은 
    + 데이터 가공/변환(전처리)
    + 모델 학습/예측
    + 평가의 과정을 거침
* 앞의 타이타닉 예제에서 모델의 평가는 **정확도**만 사용했음
* 한편, 머신러닝의 예측성능의 평가방법은 다양함
    + 회귀 - R^2, MSE평균제곱오차
    + 분류 - 혼동(오차)행렬, 크로스엔트로피, 최대우도, ROC, AUC, F1스코어

## 정확도의 함정
* 앞의 타이타닉 생존자 ML예제의 정확도는 평균 80%였음
* 그런데 정확도 지표만으로 ML 모델의 성능을 파악하기에는 다소 문제가 있음 - 왜곡의 위험
* 즉, 탐색적 분석을 시행했을때 성별을 기준으로 생존비율은 여성일 때가 더 높았음
* 따라서, 굳이 ML 알고리즘을 적용하지 않아도 성별이 여성일 경우 생존, 남성일 경우 사망이라고 예측해도 크게 무리 없음
* 단순히 성별 조건 하나만 적용해도 별거 아닌 알고리즘으로도 높은 정확도가 나타나는 상황 발생

### 가짜 분류기 생성
* 성별이 남자면 사망
* 성별이 여자면 생존이라고 예측하는 더미분류기 생성

In [1]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    # 아무것도 학습하지 않는 fit 메서드 정의
    def fit(self, X, y=None):
        pass

    # 성별이 1(남성)이면 0(사망), 0이면 1
    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))
        # 입력데이터 크기만큼 0으로 채워진 1차원 행렬 생성

        for i in range(X.shape[0]):
            if X['gender'].iloc[i] != 1:
                pred[i] = 1
            # 성별이 여성인 경우 1로 설정

        return pred

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [5]:
titanic = pd.read_csv('data/titanic2.csv')

data = titanic.iloc[:, :7]
target = titanic.survived

In [9]:
X_train, X_test, Y_train,Y_test = train_test_split(data,target, train_size=0.7, stratify=target)

In [10]:
dtclf = MyDummyClassifier()

dtclf.fit(X_train, Y_train)
pred = dtclf.predict(X_test)

accuracy_score(Y_test,pred)

0.7908163265306123

In [ ]:
# 즉, 이렇게 단순한 알고리즘만으로 예측하더라도
# 데이터의 구성에 따라 정확도가 약 76%가 나옴
# 불균형한 레이블의 비율()에서는 
# 정확도 지표만으로 모델 성능을 평가하는 것은 올바르지 않음

# 예를 들어, 100개의 종속변수 중 90개의 레이블 0이고, 10개의 레이블이 1인 경우, 
# 무조건 0으로 예측결과를 반환하는 머신러닝 알고리즘의 정확도는 90%임!
# 즉, 선생님이 시험 정답지의 답을 1로만 작성한 경우, 
# 학생이 1로 찍기만 해도 100점을 맞는 경우와 비슷한 상황임!

# 따라서, 불균형한 레이블 데이터세트의 성능수치로 
# 정확도 평가지표를 사용하면 안됨
# 이러한 한계를 극복하기 위해 오차행렬 사용
# 특히, 정확도보다는 정밀도, 재현율을 더 선호

# 오차행렬
<img src='img/confusion_matrix.png' width='%'>

In [12]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test,pred)

array([[202,  41],
       [ 41, 108]], dtype=int64)